In [ ]:
"""
install the upgraded givernylocal package.
    - n.b. this only needs to be run once.
"""
!pip install --upgrade givernylocal

<hr style = "height:6px;border:none;background-color:sienna">

<h2 style = "font-weight:bold;font-style:italic">
    Getdata demo notebook
</h2>

<p style = "font-weight:bold;font-size:13px">
    &emsp;n.b. requires python 3.9+
</p>

<p style = "font-weight:bold;font-size:13px">
    <u>purpose</u> :
    <br>
    &emsp;- local processing of JHTDB datasets.
    <br><br>
    <u>supported datasets</u> :
</p>

<ul>
    <li style = "font-weight:bold;font-size:13px">
        isotropic1024coarse &ensp;: &ensp;isotropic 1024-cube (coarse).
    </li>
    <li style = "font-weight:bold;font-size:13px">
        isotropic1024fine &ensp;: &ensp;isotropic 1024-cube (fine).
    </li>
    <li style = "font-weight:bold;font-size:13px">
        isotropic4096 &ensp;: &ensp;isotropic 4096-cube.
    </li>
    <li style = "font-weight:bold;font-size:13px">
        isotropic8192 &ensp;: &ensp;isotropic 8192-cube.
    </li>
    <li style = "font-weight:bold;font-size:13px">
        sabl2048low &ensp;: &ensp;stable atmospheric boundary layer 2048-cube, low-rate timestep.
    </li>
    <li style = "font-weight:bold;font-size:13px">
        sabl2048high &ensp;: &ensp;stable atmospheric boundary layer 2048-cube, high-rate timestep.
    </li>
    <li style = "font-weight:bold;font-size:13px">
        rotstrat4096 &ensp;: &ensp;rotating stratified 4096-cube.
    </li>
    <li style = "font-weight:bold;font-size:13px">
        mhd1024 &ensp;: &ensp;magneto-hydrodynamic isotropic 1024-cube.
    </li>
    <li style = "font-weight:bold;font-size:13px">
        mixing &ensp;: &ensp;homogeneous buoyancy driven 1024-cube.
    </li>
    <li style = "font-weight:bold;font-size:13px">
        channel &ensp;: &ensp;channel flow.
    </li>
    <li style = "font-weight:bold;font-size:13px">
        channel5200 &ensp;: &ensp;channel flow (reynolds number 5200).
    </li>
    <li style = "font-weight:bold;font-size:13px">
        transition_bl &ensp;: &ensp;transitional boundary layer.
    </li>
</ul>

<p style = "font-weight:bold;font-size:13px">
    <u>functions</u> :
</p>

<ul>
    <li style = "font-weight:bold;font-size:13px">
        getData &ensp;: &ensp;retrieve (interpolate and/or differentiate) field data on a set of specified spatial points for the specified variable.
    </li>
    <li style = "font-weight:bold;font-size:13px">
        write_interpolation_tsv_file &ensp;: &ensp;write getData results to a .tsv file.
    </li>
</ul>

<hr style = "height:6px;border:none;background-color:sienna">

<h4 style = "font-weight:bold;font-style:italic">
    instantiate dataset
</h4>

<p style = "font-weight:bold;font-size:13px">
    <u>purpose</u> : 
    <br>
    &emsp;- instantiate the dataset and cache the metadata.
    <br>

</p>

<p style = "font-weight:bold;font-size:13px">
    <u>parameters</u> :
</p>

<ul>
    <li style = "font-weight:bold;font-size:13px">
        auth_token &ensp;: &ensp;turbulence user authorization token.
    </li>
    <li style = "font-weight:bold;font-size:13px">
        dataset_title &ensp;: &ensp;name of the turbulence dataset.
    </li>
    <li style = "font-weight:bold;font-size:13px">
        output_path &ensp;: &ensp;folder path where output files will be to saved to.
        <br>
        &emsp;- n.b. &ensp;: &ensp;cannot be left blank.
    </li>
</ul>

<hr style = "height:6px;border:none;background-color:sienna">

In [ ]:
"""
instantiate dataset
"""
from givernylocal.turbulence_dataset import *
from givernylocal.turbulence_toolkit import *

auth_token = 'edu.jhu.pha.turbulence.testing-201406'
dataset_title = 'channel'
output_path = './giverny_output'

# instantiate the dataset.
dataset = turb_dataset(dataset_title = dataset_title, output_path = output_path, auth_token = auth_token)

<hr style = "height:6px;border:none;background-color:sienna">

<h4 style = "font-weight:bold;font-style:italic">
    getData
</h4>

<p style = "font-weight:bold;font-size:13px">
    <u>purpose</u> : 
    <br>
    &emsp;- retrieve (interpolate and/or differentiate) a group of sparse data points.
    <br>
</p>

<p style = "font-weight:bold;font-size:13px">
    <u>steps</u> :
</p>

<ul>
    <li style = "font-weight:bold;font-size:13px">
        step 1 &ensp;: &ensp;identify the database files to be read.
    </li>
    <li style = "font-weight:bold;font-size:13px">
        step 2 &ensp;: &ensp;read the database files and store the interpolated points in an array.
    </li>
</ul>

<p style = "font-weight:bold;font-size:13px">
    <u>parameters</u> :
</p>

<ul>
    <li style = "font-weight:bold;font-size:13px">
         dataset &ensp;: &ensp;the instantiated dataset.
    </li>
    <li style = "font-weight:bold;font-size:13px">
        points &ensp;: &ensp;array of points in the domain [0, 2pi).
    </li>
    <li style = "font-weight:bold;font-size:13px">
        variable &ensp;: &ensp;type of data (velocity, pressure, energy, temperature, force, magneticfield, vectorpotential, density, position).
    </li>
    <li style = "font-weight:bold;font-size:13px">
        time &ensp;: &ensp;time (snapshot number for datasets without a full time evolution).
    </li>
    <li style = "font-weight:bold;font-size:13px">
        time_end &ensp;: &ensp;ending time for 'position' variable and time series queries.
    </li>
    <li style = "font-weight:bold;font-size:13px">
        delta_t &ensp;: &ensp;time step for 'position' variable and time series queries.
    </li>
    <li style = "font-weight:bold;font-size:13px">
        temporal_method &ensp;: &ensp;temporal interpolation methods.
        <br>
        &emsp;- none &ensp;: &ensp;No temporal interpolation (the value at the closest stored time will be returned).
        <br>
        &emsp;- pchip &ensp;: &ensp;Piecewise Cubic Hermite Interpolation Polynomial method is used, in which the value from the two nearest times<br>
        &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;is interpolated at time t using Cubic Hermite Interpolation Polynomial, with centered finite difference evaluation of the<br>
        &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;end-point time derivatives (i.e. a total of four temporal points are used).
        <br>
    </li>
    <li style = "font-weight:bold;font-size:13px">
        spatial_method &ensp;: &ensp;spatial interpolation and differentiation methods.
        <br>
        &emsp;- none &ensp;: &ensp;No spatial interpolation (value at the datapoint closest to each coordinate value).
        <br>
        &emsp;- lag4 &ensp;: &ensp;4th-order Lagrange Polynomial interpolation along each spatial direction.
        <br>
        &emsp;- lag6 &ensp;: &ensp;6th-order Lagrange Polynomial interpolation along each spatial direction.
        <br>
        &emsp;- lag8 &ensp;: &ensp;8th-order Lagrange Polynomial interpolation along each spatial direction.
        <br>
        &emsp;- m1q4 &ensp;: &ensp;Splines with smoothness 1 (3rd order) over 4 data points.
        <br>
        &emsp;- m2q8 &ensp;: &ensp;Splines with smoothness 2 (5th order) over 8 data points. 
        <br>
        &emsp;- m2q14 &ensp;: &ensp;Splines with smoothness 2 (5th order) over 14 data points. 
        <br>
        &emsp;- fd4noint &ensp;: &ensp;4th-order centered finite differencing (without spatial interpolation).
        <br>
        &emsp;- fd6noint &ensp;: &ensp;6th-order centered finite differencing (without spatial interpolation).
        <br>
        &emsp;- fd8noint &ensp;: &ensp;8th-order centered finite differencing (without spatial interpolation).
        <br>
        &emsp;- fd4lag4 &ensp;: &ensp;4th-order Lagrange Polynomial interpolation in each direction, of the 4th-order finite difference values on the grid.
        <br>
    </li>
    <li style = "font-weight:bold;font-size:13px">
        spatial_operator &ensp;: &ensp;spatial interpolation and differentiation operator.
        <br>
        &emsp;- field &ensp;: &ensp;function evaluation &amp; interpolation.
        <br>
        &emsp;- gradient &ensp;: &ensp;differentiation &amp; interpolation.
        <br>
        &emsp;- hessian &ensp;: &ensp;differentiation &amp; interpolation.
        <br>
        &emsp;- laplacian &ensp;: &ensp;differentiation &amp; interpolation.
        <br>
    </li>
</ul>

<p style = "font-weight:bold;font-size:13px">
    <u>output</u> :
</p>

<ul>
    <li style = "font-weight:bold;font-size:13px">
        result &ensp;: &ensp;interpolated/differentiated values array.
    </li>
</ul>

<hr style = "height:6px;border:none;background-color:sienna">

In [ ]:
"""
initialize getData parameters (except time and points)
"""
variable = 'velocity'
temporal_method = 'none'
spatial_method = 'lag8'
spatial_operator = 'field'

"""
example point distributions (2D plane, 3D box, random, time series) are provided below...
""";

In [ ]:
"""
2D plane demo points : evenly spaced over a 2D plane lying along one of the primary axes
    - time : the time to be queried (snapshot number for datasets without a full time evolution).
    - nx, nz : number of points along each axis. total number of points queried will be n_points = nx * nz.
    - x_points, y_points, z_points : point distributions along each axis, evenly spaced over the specified ranges.
        - np.linspace(axis minimum, axis maximum, number of points).
    - points : the points array evenly spaced out over the 2D plane.
        - points array is instantiated as an empty array that will be filled inside the for loops.
"""
time = 1.0

nx = 64
nz = 64
n_points = nx * nz

x_points = np.linspace(0.0, 0.4 * np.pi, nx, dtype = np.float64)
y_points = 0.9
z_points = np.linspace(0.0, 0.15 * np.pi, nz, dtype = np.float64)
        
points = np.zeros((n_points, 3), dtype = np.float64)
for i in range(nx):
    for j in range(nz):
        points[i * nz + j, 0] = x_points[i]  
        points[i * nz + j, 1] = y_points
        points[i * nz + j, 2] = z_points[j]

"""
use the tools and processing gizmos.
"""
# process interpolation/differentiation of points.
result = getData(dataset, variable, time, temporal_method, spatial_method, spatial_operator, points)

"""
display sample results on screen.
"""
# set threshold for the number of numpy array elements to display.
np.set_printoptions(threshold = 10)

print(f'num points = {len(points)}')
print(f'\npoints = \n-\n{points}')
# the 1st time index of result corresponds to the final time for the "position" variable and the initial time for all other variables.
print(f'\nresult (1st time index) = \n-\n{result[0]}\n')

"""
with result, generate a 2D contour plot.
    - a simple plot to quickly visualize the queried 2D plane.
""";
if variable != 'position' and nx >= 2 and nz >= 2:
    # user-defined plot parameters.
    # which time of the data to plot (0-based index, so the first time component is specified as 0).
    time_component = 0
    # which component (column) of the data to plot (0-based index, so the first component is specified as 0).
    plot_component = 0

    # reformat 'result' for plotting.
    result_time = result[time_component]
    result_array = np.array(result_time)
    data_plot = np.zeros((nx, nz, len(result_time.columns)), dtype = np.float32)
    x_plot = np.zeros((nx, nz), dtype = np.float64)
    z_plot = np.zeros((nx, nz), dtype = np.float64)

    for i in range(nx):
        for j in range(nz):
            x_plot[i, j] = x_points[i]
            z_plot[i, j] = z_points[j]
            data_plot[i, j, :] = result_array[i * nz + j, :]

    # plot the data.
    fig = plt.figure(figsize = (7, 7), dpi = 67)
    contour = plt.contourf(x_plot, z_plot, data_plot[:, :, plot_component], levels = 500, cmap = 'inferno')
    plt.gca().set_aspect('equal')
    # select the colorbar orientation depending on which axis is larger.
    colorbar_orientation = 'vertical' if (np.max(z_plot) - np.min(z_plot)) >= (np.max(x_plot) - np.min(x_plot)) else 'horizontal'
    cbar = plt.colorbar(contour, shrink = 0.67, orientation = colorbar_orientation)
    # rotate the horizontal colorbar labels.
    if colorbar_orientation == 'horizontal':
        for label in cbar.ax.get_xticklabels():
            label.set_rotation(90)
    cbar.set_label(f'{variable} {spatial_operator}', labelpad = 12, fontsize = 14)
    plt.title(f'{dataset_title}', fontsize = 16)
    plt.xlabel('x', labelpad = 7, fontsize = 14)
    plt.ylabel('z', labelpad = 7, fontsize = 14)
    cbar.ax.tick_params(labelsize = 12)
    plt.tick_params(axis = 'both', labelsize = 12)
    plt.tight_layout()
    plt.show()
    plt.close()

In [ ]:
"""
3D box demo points : evenly spaced over a 3D volume
    - time : the time to be queried (snapshot number for datasets without a full time evolution).
    - nx,ny,nz : number of points along each axis. total number of points queried will be n_points= nx * ny * nz.
    - x_points, y_points, z_points : point distributions along each axis, evenly spaced over the specified ranges.
        - np.linspace(axis minimum, axis maximum, number of points).
    - points : the points array evenly spaced out over the 3D volume.
        - points array is instantiated as an empty array that will be filled inside the for loops.
"""
time = 1.0

nx = 16
ny = 16
nz = 16
n_points = nx * ny * nz

x_points = np.linspace(3.0, 3.3, nx, dtype = np.float64)
y_points = np.linspace(-0.9, -0.6, ny, dtype = np.float64)
z_points = np.linspace(0.2, 0.5, nz, dtype = np.float64)

points = np.zeros((n_points, 3), dtype = np.float64)
for i in range(nx):
    for j in range(ny):
        for k in range(nz):      
            points[i * ny * nz + j * nz + k, 0] = x_points[i]  
            points[i * ny * nz + j * nz + k, 1] = y_points[j]
            points[i * ny * nz + j * nz + k, 2] = z_points[k]

"""
use the tools and processing gizmos.
"""
# process interpolation/differentiation of points.
result = getData(dataset, variable, time, temporal_method, spatial_method, spatial_operator, points)

"""
display sample results on screen.
"""
# set threshold for the number of numpy array elements to display.
np.set_printoptions(threshold = 10)

print(f'num points = {len(points)}')
print(f'\npoints = \n-\n{points}')
# the 1st time index of result corresponds to the final time for the "position" variable and the initial time for all other variables.
print(f'\nresult (1st time index) = \n-\n{result[0]}\n')

"""
with result, generate a 2D contour plot.
    - a simple plot to quickly visualize the queried 3D volume on a sample plane at y.
""";
if variable != 'position' and nx >= 2 and nz >= 2:
    # user-defined plot parameters.
    # which time of the data to plot (0-based index, so the first time component is specified as 0).
    time_component = 0
    # which component (column) of the data to plot (0-based index, so the first component is specified as 0).
    plot_component = 0
    
    # reformat 'result' for plotting.
    result_time = result[time_component]
    result_array = np.array(result_time)
    data_plot = np.zeros((nx, ny, nz, len(result_time.columns)), dtype = np.float32)
    x_plot = np.zeros((nx, ny, nz), dtype = np.float64)
    y_plot = np.zeros((nx, ny, nz), dtype = np.float64)
    z_plot = np.zeros((nx, ny, nz), dtype = np.float64)

    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                x_plot[i, j, k] = x_points[i]
                y_plot[i, j, k] = y_points[j]
                z_plot[i, j, k] = z_points[k]
                data_plot[i, j, k, :] = result_array[i * ny * nz + j * nz + k, :]

    # plot the data.
    fig = go.Figure(
        data = go.Volume(
            x = x_plot.flatten(), y = y_plot.flatten(), z = z_plot.flatten(),
            value = data_plot[:, :, :, plot_component].flatten(),
            isomin = data_plot[:, :, :, plot_component].min(),
            isomax = data_plot[:, :, :, plot_component].max(),
            opacity = 0.2, surface_count = 17, colorscale = 'inferno',
            colorbar = dict(
                title = dict(text = f'{variable} {spatial_operator}', side = 'right', font = dict(size = 14)),
                thickness = 15, len = 0.75,
                x = 1.05, y = 0.5, xanchor = 'left', yanchor = 'middle'),
            showlegend = False))

    # update figure layout.
    fig.update_layout(
        title = dict(
            text = f'{dataset_title} (click and drag to rotate)', x = 0.5, y = 0.95, xanchor = 'center', yanchor = 'top', font = dict(size = 16)),
        scene = dict(
            xaxis = dict(title = 'x', gridcolor = 'lightgrey', showline = True, linecolor = 'black'),
            yaxis = dict(title = 'y', gridcolor = 'lightgrey', showline = True, linecolor = 'black'),
            zaxis = dict(title = 'z', gridcolor = 'lightgrey', showline = True, linecolor = 'black'),
            aspectmode = 'data',
            camera = dict(eye = dict(x = 1.5, y = 1.5, z = 1.5)),
            bgcolor = 'white'),
        width = 600, height = 600, plot_bgcolor = 'white', paper_bgcolor = 'white',
        showlegend = False)

    fig.show()

In [ ]:
"""
random box demo points : uniformly distributed over the specified domain
    - time : the time to be queried (snapshot number for datasets without a full time evolution).
    - n_points : number of points.
    - min_xyz, max_xyz : minimum and maximum (x, y, z) axes boundaries for generating the random distribution of points within.
    - points : the points array containing a random distribution of points in the specified domain.
"""
time = 1.0

n_points = 1000

min_xyz = [6.1359, -0.61359, 0.6]
max_xyz = [21.8656, 0.8656, 8.8656]

points = np.random.uniform(low = [min_xyz[0], min_xyz[1], min_xyz[2]],
                           high = [max_xyz[0], max_xyz[1], max_xyz[2]],
                           size = (n_points, 3))

"""
use the tools and processing gizmos.
"""
# process interpolation/differentiation of points.
result = getData(dataset, variable, time, temporal_method, spatial_method, spatial_operator, points)

"""
display sample results on screen.
"""
# set threshold for the number of numpy array elements to display.
np.set_printoptions(threshold = 10)

print(f'num points = {len(points)}')
print(f'\npoints = \n-\n{points}')
# the 1st time index of result corresponds to the final time for the "position" variable and the initial time for all other variables.
print(f'\nresult (1st time index) = \n-\n{result[0]}\n')

"""
with result, generate a 3D scatter plot and histogram.
    - simple plots to quickly visualize the queried box volume.
""";
if variable != 'position':
    # user-defined plot parameters.
    # which time of the data to plot (0-based index, so the first time component is specified as 0).
    time_component = 0
    # which component (column) of the data to plot (0-based index, so the first component is specified as 0).
    plot_component = 0
    # number of bins for the histogram.
    bins = 20

    x_plot, y_plot, z_plot = points[:, 0], points[:, 1], points[:, 2]
    data_plot = np.array(result[time_component])[:, plot_component]
    
    # plot the data.
    # scatter subplot.
    scatter = go.Scatter3d(
        x = x_plot, y = y_plot, z = z_plot, mode = 'markers',
        marker = dict(
            size = 4,
            color = data_plot,
            colorscale = 'inferno',
            opacity = 0.8,
            colorbar = dict(
                title = dict(text = f'{variable} {spatial_operator}', side = 'bottom', font = dict(size = 14)),
                thickness = 15, len = 0.4, orientation = 'h',ticks = 'outside', outlinecolor = 'black', outlinewidth = 1,
                x = 0.23, y = -0.2, xanchor = 'center', yanchor = 'bottom')),
        showlegend = False)

    # histogram subplot.
    histogram = go.Histogram(
        x = data_plot, nbinsx = bins,
        marker_color = 'forestgreen', marker_line = dict(color = 'black', width = 1), opacity = 0.85,
        showlegend = False)

    # make subplots.
    fig = make_subplots(
        rows = 1, cols = 2, 
        specs = [[{'type': 'scene'}, {'type': 'xy'}]],
        subplot_titles = ('scatter (click and drag to rotate)', 'histogram'),
        column_widths = [0.47, 0.47], horizontal_spacing = 0.06)

    # add scatter and histogram traces to subplots.
    fig.add_trace(scatter, row = 1, col = 1)
    fig.add_trace(histogram, row = 1, col = 2)

    # update figure layout.
    fig.update_layout(
        title = dict(
            text = f'{dataset_title}', x = 0.5, y = 0.95, xanchor = 'center', yanchor = 'top', font = dict(size = 16)),
        scene = dict(
            xaxis = dict(title = 'x', gridcolor = 'lightgrey', showline = True, linecolor = 'black'),
            yaxis = dict(title = 'y', gridcolor = 'lightgrey', showline = True, linecolor = 'black'),
            zaxis = dict(title = 'z', gridcolor = 'lightgrey', showline = True, linecolor = 'black'),
            aspectmode = 'data',
            camera = dict(eye = dict(x = 2.5, y = 2.5, z = 2.5)),
            bgcolor = 'white'),
        xaxis_title = variable, yaxis_title = 'count',
        height = 600, width = 1200, plot_bgcolor = 'white', paper_bgcolor = 'white',
        showlegend = False)

    # display plot boundaries.
    fig.update_xaxes(showline = True, linewidth = 1, linecolor = 'black', mirror = True)
    fig.update_yaxes(showline = True, linewidth = 1, linecolor = 'black', mirror = True)

    fig.show()

In [ ]:
"""
time series demo point(s)
    - time : the start time of the time series (snapshot number for datasets without a full time evolution).
    - time_end : the end time of the time series (snapshot number for datasets without a full time evolution).
    - delta_t : time step.
    - points : the points array.
"""
# create a temporary variable for the temporal method to set to 'pchip' for this time series example.
temporal_method_tmp = 'pchip'

time = 0.1
time_end = 0.5
delta_t = 0.008
option = [time_end, delta_t]

points = np.array([[10.33, 0.9, 4.6]], dtype = np.float64)

"""
use the tools and processing gizmos.
"""
# process interpolation/differentiation of points.
result = getData(dataset, variable, time, temporal_method_tmp, spatial_method, spatial_operator, points, option)

"""
display sample results on screen.
"""
# set threshold for the number of numpy array elements to display.
np.set_printoptions(threshold = 10)

print(f'num points = {len(points)}')
print(f'\npoints = \n-\n{points}')
# the 1st time index of result corresponds to the final time for the "position" variable and the initial time for all other variables.
print(f'\nresult (1st time index) = \n-\n{result[0]}\n')

"""
with result, generate an x-y signal plot.
    - a simple plot to quickly visualize the queried point over time.
""";
if variable != 'position':
    # user-defined plot parameters.
    # which point of the data to plot (0-based index, so the first point component is specified as 0).
    point_component = 0
    # which component (column) of the data to plot (0-based index, so the first component is specified as 0).
    plot_component = 0

    times_plot = np.arange(time, time_end, delta_t)
    # append time_end if the final time in times_plot is delta_t less than time_end due to machine precision.
    if math.isclose(times_plot[-1] + delta_t, time_end, rel_tol = 10**-9, abs_tol = 0.0):
        times_plot = np.append(times_plot, time_end)
    data_plot = np.array(result)[:, point_component][:, plot_component]

    # plot the data.
    fig = plt.figure(figsize = (7, 7), dpi = 67)
    signal = plt.plot(times_plot, data_plot, color = 'forestgreen', linewidth = 3)
    plt.title(f'{dataset_title}, time series at xyz = ({points[0][0]:.2f}, {points[0][1]:.2f}, {points[0][2]:.2f})', fontsize = 16)
    plt.xlabel(f'time', labelpad = 7, fontsize = 14)
    plt.ylabel(f'{variable}', labelpad = 7, fontsize = 14)
    plt.tick_params(axis = 'both', labelsize = 12)
    plt.tight_layout()
    plt.show()
    plt.close()

<hr style = "height:6px;border:none;background-color:sienna">

<h4 style = "font-weight:bold;font-style:italic">
    save interpolation results
</h4>

<p style = "font-weight:bold;font-size:13px">
    <u>purpose</u> : 
    <br>
    &emsp;- save the interpolated/differentiated points retrieved by the getData function.
    <br>
</p>

<p style = "font-weight:bold;font-size:13px">
    <u>parameters</u> :
</p>

<ul>
    <li style = "font-weight:bold;font-size:13px">
         dataset &ensp;: &ensp;the instantiated dataset.
    </li>
    <li style = "font-weight:bold;font-size:13px">
         points &ensp;: &ensp;input points to getData.
    </li>
    <li style = "font-weight:bold;font-size:13px">
         result &ensp;: &ensp;output from getData.
    </li>
    <li style = "font-weight:bold;font-size:13px">
        output_filename &ensp;: &ensp;filename for the tsv file to be saved in the output_path folder.
    </li>
</ul>

<p style = "font-weight:bold;font-size:13px">
    <u>output</u> :
</p>

<ul>
    <li style = "font-weight:bold;font-size:13px">
        tsv file &ensp;: &ensp;saved to output_filename in the output_path folder.
    </li>
</ul>

<hr style = "height:6px;border:none;background-color:sienna">

In [ ]:
"""
write the interpolation results to a tab-separated value (tsv) file
"""
output_filename = 'turbulence-interpolation'

"""
use the tsv writing gizmo.
"""
# write the tsv output file.
write_interpolation_tsv_file(dataset, points, result, output_filename)